In [1]:
print("Hello LangChain!!")

Hello LangChain!!


In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

#load_dotenv(dotenv_path='.env')
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])


gsk_Z


In [3]:
# prompt + llm + output 

# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(type(prompt))
print(prompt)

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [4]:
prompt_text = prompt.format(input="파이썬은 무엇인가요? 자세하게 설명해주세요")
print(type(prompt_text))
print(prompt_text)

<class 'str'>
System: 당신은 개발자입니다.
Human: 파이썬은 무엇인가요? 자세하게 설명해주세요


In [5]:
#llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125")

# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="moonshotai/kimi-k2-instruct-0905",
    model="openai/gpt-oss-120b",
    temperature=0.7
)
print(type(llm))
print(llm)

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
client=<openai.resources.chat.completions.completions.Completions object at 0x0000021A6EF568A0> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021A706A3CB0> root_client=<openai.OpenAI object at 0x0000021A703E0950> root_async_client=<openai.AsyncOpenAI object at 0x0000021A704631A0> model_name='openai/gpt-oss-120b' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [6]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
응답: ## 파이썬(Python)이란?

파이썬은 **고수준(high‑level) 인터프리터 언어**이며, **동적 타이핑(dynamic typing)**과 **자동 메모리 관리(garbage collection)**를 제공하는 **범용 프로그래밍 언어**입니다. 1991년 네덜란드의 귀도 반 로섬(Guido van Rossum)이 처음 발표했으며, 현재는 전 세계 개발자 커뮤니티와 기업에서 가장 많이 사용되는 언어 중 하나입니다.  

아래에서는 파이썬의 **역사·특징·구조·주요 활용 분야·생태계**를 상세히 살펴보겠습니다.

---

## 1. 파이썬의 역사와 발전

| 연도 | 주요 사건 |
|------|-----------|
| **1980년대 후반** | 귀도 반 로섬이 “ABC” 언어의 개선판을 구상. |
| **1991년** | 파이썬 0.9.0 공개 (유닉스, 윈도우, 매크로 언어). |
| **1994년** | 파이썬 1.0 발표 – 모듈, 예외 처리, 함수형 프로그래밍 요소 추가. |
| **2000년** | 파이썬 2.0 출시 – 리스트 컴프리헨션, 가비지 컬렉션, 유니코드 지원. |
| **2008년** | 파이썬 3.0 (Python 3000) 발표 – 문자열/바이트 구분, `print` 함수화 등 비호환적 변화. |
| **2020년** | 파이썬 3.9, 3.10, 3.11 등 최신 버전에서 성능 개선(패턴 매칭, JIT-like 최적화) 및 새로운 표준 라이브러리 추가. |
| **2023년** | 파이썬 3.12 발표 – 파싱 속도 2배 이상 향상, 새로운 `tomllib` 등. |
| **2025년 현재** | 파이썬 3.13이 베타 단계에 있으며, **CPython**(C 구현)과 **PyPy**(JIT 구현), **MicroPython**, **RustPython** 등 다양한 구현체가 활발히 유지·보수 중. |

---

## 2. 파이썬

# LCEL

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser
print(type(chain))
print(chain)

<class 'langchain_core.runnables.base.RunnableSequence'>
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000021A6EF568A0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000021A706A3CB0>, root_client=<openai.OpenAI object at 0x0000021A703E0950>, root_async_client=<openai.AsyncOpenAI object at 0x0000021A704631A0>, model_name='openai/gpt-oss-120b', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')] last=StrOutputPars

In [9]:
response = chain.invoke({"input":"LangChain은 무엇인가요?"})

In [10]:
print(type(response))
print(response)

<class 'str'>
**LangChain(랭체인)**은 **대규모 언어 모델(LLM)**을 **애플리케이션 수준**으로 활용할 수 있게 도와주는 **오픈소스 프레임워크**입니다.  
LLM 자체는 “텍스트를 생성·예측”하는 엔진에 불과하지만, 실제 서비스에서는 다음과 같은 복합적인 작업 흐름이 필요합니다.

| 필요 기능 | LangChain이 제공하는 핵심 컴포넌트 |
|----------|-----------------------------------|
| **프롬프트 관리** | `PromptTemplate`, `FewShotPromptTemplate` 등으로 재사용 가능한 프롬프트를 정의·조합 |
| **체인(Chain)** | 여러 LLM 호출·전처리·후처리를 순차·조건부로 연결 (예: `LLMChain`, `SequentialChain`, `RouterChain`) |
| **에이전트(Agent)** | 목표에 맞는 도구(tool)를 동적으로 선택·실행 (예: `ZeroShotAgent`, `ReactAgent`) |
| **메모리(Memory)** | 대화·작업 이력 보관 (예: `ConversationBufferMemory`, `VectorStoreRetrieverMemory`) |
| **도구·통합** | 검색 엔진, 데이터베이스, 파일 시스템, API 등 외부 서비스와 연결 (예: `Tool`, `Retriever`) |
| **벡터 스토어** | 임베딩 기반 검색·유사도 매칭 (FAISS, Pinecone, Weaviate 등) |
| **배포·운영** | `LangChainHub`(프롬프트·체인 공유), `LangServe`(REST API로 쉽게 배포) 등 |

---

## 1. 왜 LangChain이 필요한가?

1. **LLM을 단일 호출이 아닌 파이프라인**으로 사용하고 싶다.  
2. **프롬프트를 체계적으로 관리·버전 관리**하고 싶다.  
3. **외부 데이터(검색 결과, DB, 파일 등)와 결합**한 복합 작업을 구현